<a href="https://colab.research.google.com/github/JuanPR27/ElectivaTecnicaI/blob/main/SQLite_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gestión Básica de Inventario con SQLite3**

**Autor: Juan Manuel Preciado Rojas**

Bgotá, Colombia

Actualizado: 9 octubre 2025

Nota: Código ajustado del proporcionado por el Profesor

###Instalar librerías SQLite3

In [ ]:
%%shell
sudo apt-get install sqlite3

###Crear la base de datos y una tabla de inventario

In [ ]:
import sqlite3
import random

#Crear la base de datos en SQLite
conn = sqlite3.connect('productos.db')
c = conn.cursor()

In [ ]:
#Crear una tabla llamada 'inventario'
c.execute('''
CREATE TABLE inventario (
    id INTEGER PRIMARY KEY,
    nombre TEXT,
    categoria TEXT,
    precio DECIMAL(10, 2),
    cantidad INTEGER,
    proveedor TEXT,
    fecha_ingreso DATE,
    ubicacion TEXT
)
''')

###Generar datos de ejemplo para la tabla

In [ ]:
def generar_producto(id):
    nombres = ["Producto", "Artículo", "Objeto"]
    categorias = ["Electrónica", "Hogar", "Jardín", "Moda", "Deportes"]
    proveedores = ["Proveedor1", "Proveedor2", "Proveedor3"]
    ubicaciones = ["Almacén A", "Almacén B", "Almacén C"]

    nombre = random.choice(nombres) + str(id)
    categoria = random.choice(categorias)
    precio = round(random.uniform(10.0, 500.0), 2)
    cantidad = random.randint(1, 100)
    proveedor = random.choice(proveedores)
    fecha_ingreso = f"2023-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}"
    ubicacion = random.choice(ubicaciones)

    return (id, nombre, categoria, precio, cantidad, proveedor, fecha_ingreso, ubicacion)

# Insertar 1000 productos en la tabla
productos = [generar_producto(i + 1) for i in range(10000)]
c.executemany('INSERT INTO inventario VALUES (?, ?, ?, ?, ?, ?, ?, ?)', productos)

# Guardar cambios y cerrar la conexión
conn.commit()
conn.close()

print("Base de datos creada y poblada con éxito.")

###Consulta para mostrar todos los productos creados

In [ ]:
import sqlite3

# Conectar a la base de datos
conn = sqlite3.connect('productos.db')
c = conn.cursor()

# Consultar todos los productos
c.execute('SELECT * FROM inventario')
productos = c.fetchall()

# Imprimir los productos
for producto in productos:
    print(producto)

# Cerrar la conexión
conn.close()

###Realizar consulta y transferir a dataframe

In [ ]:
import sqlite3
import pandas as pd

# Conectar a la base de datos
conn = sqlite3.connect('productos.db')

# Realizar la consulta y cargar los resultados en un DataFrame
query = "SELECT * FROM inventario WHERE precio > 450"
df_productos = pd.read_sql_query(query, conn)

# Mostrar el DataFrame
print(df_productos)

# Cerrar la conexión
conn.close()

In [ ]:
df_productos.head()

###Nueva consulta

In [ ]:
import sqlite3
import pandas as pd

# Conectar a la base de datos
conn = sqlite3.connect('productos.db')

# Realizar la consulta y cargar los resultados en un DataFrame
query = "SELECT * FROM inventario WHERE precio > 480 OR cantidad < 5"
df_productos = pd.read_sql_query(query, conn)

# Mostrar el DataFrame
print(df_productos)


# Cerrar la conexión
conn.close()

###Insertar nuevo producto

In [ ]:
# Datos del nuevo producto
nuevo_producto = (
    None,  # SQLite genera un nuevo id automáticamente si la columna es autoincrement
    'NuevoProducto123',
    'Electrónica',
    249.99,
    20,
    'ProveedorX',
    '2024-04-16',
    'Almacén D'
)

# Conexión con la DB
conn = sqlite3.connect('productos.db')
c = conn.cursor()

# Insersión de nuevo producto
c.execute('INSERT INTO inventario VALUES (?, ?, ?, ?, ?, ?, ?, ?)', nuevo_producto)

# Guardar (commit) los cambios y cerrar conexión con la DB
conn.commit()
conn.close()

##Insertar nuevo producto desde Excel

###Crear tabla excel de nuevos productos

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

np.random.seed(0)
num_prods = 5000

# Productos nuevos
nombres = ['Producto' + str(i) for i in range(num_prods)]
categorias = np.random.choice(['Electrónica', 'Hogar', 'Jardín', 'Moda', 'Deportes'], num_prods)
precios = np.round(np.random.uniform(10.0, 500.0, num_prods), 2)
cantidades = np.random.randint(1, 100, num_prods)
proveedores = np.random.choice(['Proveedor1', 'Proveedor2', 'Proveedor3'], num_prods)
fechas = [datetime.today() - timedelta(days=np.random.randint(0, 365)) for _ in range(num_prods)]
ubicaciones = np.random.choice(['Almacén A', 'Almacén B', 'Almacén C'], num_prods)

# DataFrame
df = pd.DataFrame({
    'Nombre': nombres,
    'Categoría': categorias,
    'Precio': precios,
    'Cantidad': cantidades,
    'Proveedor': proveedores,
    'Fecha de Ingreso': pd.to_datetime(fechas),
    'Ubicación': ubicaciones
})

print(df.head())

# Guardar el DataFrame en un archivo Excel
df.to_excel('nuevos_productos_inventario.xlsx', index=False)

###Agregar datos de tabla excelal inventario

In [ ]:
import pandas as pd

# Leer archivo Excel
ruta_archivo = '/content/nuevos_productos_inventario.xlsx'
df = pd.read_excel(ruta_archivo)

# Verificación de formatos (tipo de dato)
# Precio a float
if df['Precio'].dtype != 'float':
    df['Precio'] = df['Precio'].astype(float)

# Fecha de Ingreso a datetime
if df['Fecha de Ingreso'].dtype != 'datetime64[ns]':
    df['Fecha de Ingreso'] = pd.to_datetime(df['Fecha de Ingreso'], errors='coerce')

# Verificar y corregir fechas no válidas, usando la fecha de hoy
df['Fecha de Ingreso'].fillna(pd.Timestamp.today(), inplace=True)

df.head()

In [ ]:
import sqlite3

# Conexión con la DB
conn = sqlite3.connect('productos.db')
c = conn.cursor()

# Insersión de datos en la DB
for index, row in df.iterrows():
    c.execute("""
        INSERT INTO inventario (id, nombre, categoria, precio, cantidad, proveedor, fecha_ingreso, ubicacion)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)""", (None, row['Nombre'], row['Categoría'], row['Precio'], row['Cantidad'], row['Proveedor'], row['Fecha de Ingreso'], row['Ubicación']))


# # Insersión de datos en la DB
# for index, row in df.iterrows():
#     c.executemany("""
#         INSERT INTO inventario (nombre, categoria, precio, cantidad, proveedor, fecha_ingreso, ubicacion)
#         VALUES(?, ?, ?, ?, ?, ?,)""",
#         [row['Nombre'], row['Categoría'], row['Precio'], row['Cantidad'], row['Proveedor'], row['Fecha de Ingreso'], row['Ubicación']])

# Guardar cambios y cerrar conexión con DB
conn.commit()
conn.close()

print("¡Datos importados exitosamente!")